In [1]:
import numpy as np
from pymbar import bar, timeseries
from saltswap import analysis_tools as at

## Free energies from SAMS binary update schemes

In [2]:
deltafs_sams = np.array([317.006, 317.583, 316.958])
mu = deltafs_sams.mean()
stderr = 1.96 * deltafs_sams.std() / np.sqrt(len(deltafs_sams))
print('SAMS Delta F = {0:.2f} +/- {1:.2f} '.format(mu, stderr))

SAMS Delta F = 317.18 +/- 0.32 


## Free energies from NCMC work update schemes

In the `swapper` acceptance test the proposal to select water and salt for creation and deletion is accounted for. These can be considered as part of the work when comparing the to SAMS free energies. 

In [3]:
def add_salt_factor(nwats, nsalt):
    """
    The proposal probability for selecting two water molecules to swap with an anion and cation. 
    """
    return np.log(1.0 * nwats * (nwats - 1) / (nsalt + 1) / (nsalt + 1))

def remove_salt_factor(nwats, nsalt):
    """
    The proposal probability for selecting an anion and cation and swapping with two water molecules. 
    """
    return np.log(1.0 * nsalt * nsalt / (nwats + 1) / (nwats + 2))

In [4]:
nwats = 887  # The number of waters in these simulations

deltafs_bar = []
for i in (1, 2, 3):
    work_add = at.read_work('work_add_data{0}.txt'.format(i))
    work_rm = at.read_work('work_rm_data{0}.txt'.format(i))
    indices = timeseries.subsampleCorrelatedData(work_add)
    w_f = work_add[indices] - add_salt_factor(nwats, 0)
    indices = timeseries.subsampleCorrelatedData(work_rm)
    w_b = work_rm[indices] - remove_salt_factor(nwats - 1, 1)
    deltafs_bar.append(bar.BAR(w_f, w_b, compute_uncertainty=False))

deltafs_bar = np.array(deltafs_bar)
mu = deltafs_bar.mean()
stderr = 1.96 * deltafs_bar.std() / np.sqrt(len(deltafs_bar))
print('SAMS Delta F = {0:.2f} +/- {1:.2f} '.format(mu, stderr))

SAMS Delta F = -317.28 +/- 0.21 


These numbers are in excellent agreement with the SAMS estimates. This implies the energy calculations and acceptance tests are working as they should.